In [1]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None
sns.set_theme()

In [ ]:
train = pd.read_csv("train.csv")
len(train.filename.unique())

In [ ]:
test = pd.read_csv("test.csv")
len(test.filename.unique())

In [ ]:
test.head()

In [ ]:
def isValid(*dirs):
    return os.path.isfile(os.path.join(os.getcwd(),*dirs))
train['valid_path'] = [isValid('data','imagenes',filename) for filename in train.filename]
train.valid_path.value_counts()

In [ ]:
test['valid_path'] = [isValid('data','imagenes',filename) for filename in test.filename]
test.valid_path.value_counts()

In [ ]:
data_total = pd.concat([train,test]).reset_index(drop=True)
data_total.to_csv("etiquetas.csv", header=True, index_label=False, index = False)
sns.histplot(data = data_total, y = "class")
plt.tight_layout()

In [ ]:
data_total['valid_path'] = [isValid("data","imagenes",filename) for filename in data_total.filename]
data_total.valid_path.value_counts()

## Chequeo de imagenes y etiquetas

In [2]:
import shutil
def make_compress_folders_from_labels(dta:pd.DataFrame,column:str,make_dirs = True,compress_dirs = False):
    for label in dta[column].unique():
        if  label == None:
            label = 'Borrado'
        if make_dirs & (not os.path.exists(os.path.join(os.getcwd(), label))):
            os.makedirs(os.path.join(os.getcwd(), label))
        if compress_dirs:
            shutil.make_archive(f"{label}", "zip", os.path.join(os.getcwd(), f"{label}"))

In [3]:
from PIL import Image,ImageDraw,ImageFont
from tqdm import tqdm
def take_labels_out(data:pd.DataFrame,column:str,report_name:str=''):
    enriching_list = []
    # os.makedirs()
    for name in tqdm(data.filename.unique(),total = len(data.filename.unique())):
        image_file = Image.open(os.path.join(os.getcwd(),'data','imagenes',name))
        image_df = data.loc[data.filename == name]
        image_df['Numero de etiquetas diferentes en la imagen'] = image_df.shape[0]
        image_df['Numero de clases de etiquetas diferentes en la imagen'] = len(image_df[column].unique())
        image_df['duplicamiento de etiqueta en x'] = image_df['xmax'].duplicated(keep=False) & image_df['xmin'].duplicated(keep=False)
        image_df['duplicamiento de etiqueta en y'] = image_df['ymax'].duplicated(keep=False) & image_df['ymin'].duplicated(keep=False)
        # if 'Mascara_Encabezados' in image_df['class'].unique() : #len(image_df['class'].unique()) < 6:
        # image_df['name_with_position'] = ''
        image_df['name_of_picture'] = ''
        # print(image_df.reset_index())
        image_df = image_df.reset_index(drop = True)
        for i,row in image_df.iterrows():
            # print(i)
            if row[column] == None:
                row[column] = 'Borrado'
            image_df['name_of_picture'][i] = f'{name[:-4]}_{row[column]}_{i}.jpg'
            # image_df['name_with_position'][i] = f'{name[:-4]}_{row["class"]}_{(row["xmin"],row["ymin"],row["xmax"],row["ymax"])}_{i}.jpg'
            draw_row = ImageDraw.Draw(image_file)
            draw_row.rectangle((row['xmin'],row['ymin'],row['xmax'],row['ymax']),outline=(250,0,0))
            # font = ImageFont.truetype("sans-serif.ttf", 16)
            draw_row.text((row['xmin'],row['ymin']),row[column],(0,0,0))#,font=font)
            label_mask = image_file.crop((row['xmin'],row['ymin'],row['xmax'],row['ymax']))
            label_mask.save(os.path.join(os.getcwd(),row[column],f'{name[:-4]}_{row[column]}_{i}.jpg'),format="JPEG", quality=90)
            # image_file.save(os.path.join(os.getcwd(),'label_data', f'{name[:-4]}_highligted.jpg'), format="JPEG", quality=90)
        # print(name)
        
        # image_file.show()
        enriching_list.append(image_df)
    data_total_enrich = pd.concat(enriching_list).reset_index(drop = True)   


# print(data_total_enrich.mean(numeric_only=True).to_frame())
    cols_to_select = data_total_enrich.drop(['xmin','xmax','ymin','ymax'], axis=1).columns
    statistic_analisis = data_total_enrich[cols_to_select].describe()

    with pd.ExcelWriter(f"analisis_descriptivo_{report_name}.xlsx") as writer:
        data_total_enrich.to_excel(writer, sheet_name="base enriquecida")  
        statistic_analisis.to_excel(writer, sheet_name="estadistica descriptiva") 


## Entendiendo de la revision

In [ ]:
import pandas as pd
from PIL import Image
import time
import os
from tqdm import tqdm

In [ ]:
datos_a_cambiar = pd.read_excel("Revision imgs.xlsx")
datos_a_cambiar = datos_a_cambiar.rename(columns = {'Carpeta':'Label_Actual','Donde deberia estar':'Label_Deseado'})

In [ ]:

datos_a_cambiar['folder'] = datos_a_cambiar.Imagen.str.split('_')
datos_a_cambiar['folder'] = datos_a_cambiar['folder'].apply(lambda x: f'{x[1]}_{x[2]}')
# datos_a_cambiar['revisado'] = False
a_revisar = []
for i,row in tqdm(datos_a_cambiar.iterrows(),total=datos_a_cambiar.shape[0]):
    # print(type(row))
    # if not row['revisado']: #row['Label_Deseado'] not in datos_a_cambiar.Label_Actual.unique():
    read_path = os.path.join(os.getcwd(),row.folder,f'{row.Imagen}.jpg')
    if not os.path.isfile(read_path):
        print(row.to_frame().transpose())
        a_revisar.append(row.to_frame().transpose())
    else:
        imagen_tmp = Image.open(read_path)
        # print(row['Label_Actual'],row['Label_Deseado'])
        if row['Label_Deseado'] == '?':
            row['Label_Deseado'] = 'Desconocido'
        save_folder = os.path.join(os.getcwd(),f'{row["Label_Actual"]}_covertir_a_{row["Label_Deseado"]}')
        if not os.path.exists(save_folder):
            os.makedirs(save_folder)
        imagen_tmp.save(os.path.join(save_folder,f"{row['Imagen']}.jpg"),format="JPEG", quality=90)
            # datos_a_cambiar['revisado'][i] = True
        # time.sleep(5)
direcciones_anomalas = pd.concat(a_revisar)
direcciones_anomalas.to_csv('archivos_nombres_equivocados.csv',index = False)
    

## Chequeando por archivos faltantes

In [1]:
import os
import pandas as pd
from thefuzz import fuzz

c:\Users\JUAN PABLO\Documents\Anaconda\envs\thesis\lib\site-packages\thefuzz\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
def find_file(start_path, target_file):
    for root, dirs, files in os.walk(start_path):
        if target_file in files:
            return ['encontrado',{os.path.join(root, target_file)}]
        else:
            for file_name in files:
                partial_ratio = fuzz.partial_ratio(target_file, file_name)
                if (partial_ratio >= 97) & (partial_ratio < 100)  :
                    return ['coincidencia parcial encontrada', {os.path.join(root, file_name)}]
    return ['archivo no encontrado',None]

In [ ]:
archivos_anomalos = pd.read_csv("archivos_nombres_equivocados.csv")
archivos_anomalos['status'] = archivos_anomalos['Imagen'].apply(lambda x: find_file(os.getcwd(),x)[0])
archivos_anomalos['location'] = archivos_anomalos['Imagen'].apply(lambda x: find_file(os.getcwd(),x)[1])
archivos_anomalos.to_csv('archivos_nombres_equivocados_buscados.csv')

In [4]:
archivo_original_etiquetas = pd.read_excel("analisis_descriptivo_etiquetas.xlsx",sheet_name= 'base enriquecida')

In [5]:
archivo_original_etiquetas = archivo_original_etiquetas.rename(columns={'class':'old_class'})
archivo_original_etiquetas['new_class'] = archivo_original_etiquetas['old_class']
# archivo_original_etiquetas.head()

In [6]:
def change_label_base_on_folder_structure(*address,delete = False):
    for root,dirs,files in os.walk(os.path.join(os.getcwd(),*address)):#'por_cambiar_etiquetas','Cambio_simple'
        for directory in dirs:
            for img in os.listdir(os.path.join(root,directory)):
                directory_info = directory.split('_')
                new_class = directory_info[-1] 
                if delete:
                    new_class = None
                img_info = img.split('_')
                archivo_original_etiquetas.loc[(archivo_original_etiquetas.filename == f'{img_info[0]}.png') & (archivo_original_etiquetas.name_of_picture == img),'new_class'] = new_class
            # print(img.split('_'))
        # print(os.listdir(os.path.join(root,directory)),directory)

In [7]:
change_label_base_on_folder_structure('por_cambiar_etiquetas','Cambio_simple')
change_label_base_on_folder_structure('por_cambiar_etiquetas','borrar',delete=True)

In [ ]:
archivo_original_etiquetas.loc[archivo_original_etiquetas.name_of_picture == 'COL000001500_Mascara_Descripciones_1788.jpg']#head()

In [ ]:
archivo_original_etiquetas.loc[archivo_original_etiquetas.name_of_picture == 'COL000000745_Mascara_Descripciones_1744.jpg']

In [8]:
archivo_original_etiquetas = archivo_original_etiquetas.drop(columns=['Unnamed: 0','valid_path','Numero de etiquetas diferentes en la imagen','Numero de clases de etiquetas diferentes en la imagen','duplicamiento de etiqueta en x','duplicamiento de etiqueta en y','name_with_position'])

In [9]:
archivo_original_etiquetas.old_class = archivo_original_etiquetas.old_class.apply(lambda x: x.split('_')[-1])
archivo_original_etiquetas.new_class = archivo_original_etiquetas.new_class.apply(lambda x: x.split('_')[-1] if x != None else x)
archivo_original_etiquetas.to_csv("renombramiento_etiquetas.csv",index = False)#.head()

In [10]:
make_compress_folders_from_labels(archivo_original_etiquetas,'new_class')

In [11]:
take_labels_out(archivo_original_etiquetas,'new_class','etiquetas_corregidas')

100%|██████████| 985/985 [00:31<00:00, 31.20it/s]


In [ ]:
# jupyter nbconvert --to script understand_data.ipynb --output understanding_labels.py
